## 인공신경망(Artificial Neural Networks)

<img src="https://i.imgur.com/EtpSmaG.png" width="600">

여기서는 딥러닝 프레임워크(Framework)인 **텐서플로우(Tensorflow)**와 그 상위 API인 **케라스(Keras)**를 사용할 것입니다.<br/>
익숙한 데이터인 붓꽃(Iris) 데이터, 기본 예제인 손글씨 MNIST 예제를 통해 텐서플로우와 케라스의 사용법을 알아보겠습니다.

코드는 여러 번 반복해보고 여러 데이터에 적용해보는 것이 중요합니다.<br/>
예제 이외에도 여러 데이터를 Tensorflow 와 Keras 로 풀어본다면 익숙해질 것입니다.

### Iris 데이터 분류하기

분류 문제에서 예시로 자주 활용되는 **붓꽃(Iris) 데이터**에 신경망을 적용하여 봅시다.

전체 특성 중 2개의 특성(**`sepal length, petal length`**)만 선택하여 사용하고,<br/>
150개의 데이터 중 Setosa(50), Versicolor(50)만 추출하여 100개의 데이터에 대해서 **이진 분류(Binary classification)**를 진행해 보겠습니다.

1. **먼저 필요한 패키지와 라이브러리를 불러옵니다.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2. **시드(Seed)를 고정합니다.**

In [ ]:
np.random.seed(42) # numpy 랜덤 시드 고정
tf.random.set_seed(42) # tensorflow 랜덤 시드 고정

3. **Iris 데이터셋을 DataFrame 형태로 불러온 후 데이터 형태를 살펴봅니다.**

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

In [ ]:
df.head()

In [ ]:
df.shape

4. **Setosa, Versicolor 데이터만 추출하여 전처리해 줍니다.**

In [ ]:
label = df.iloc[0:100, 4].values # 4 번째 column의 값(=label)만 추출
label

타겟 레이블을 setosa = 0, versicolor = 1 로 변경해줍니다.

In [ ]:
label = np.where(label == 'Iris-setosa', 0, 1) # 값이 'Iris-setosa'인 경우 0으로 변경하고 아니면 1로 변경
label

5. **데이터가 어떤 분포를 가지고 있는지 시각화를 통해 알아봅시다.**

In [ ]:
features = df.iloc[0:100, [0,2]].values
features.shape

In [ ]:
plt.scatter(features[:50, 0], features[:50, 1], color='red', marker='o', label='setosa')
plt.scatter(features[50:100, 0], features[50:100, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.legend(loc='upper left')
plt.show()

5. **학습 데이터셋(Train Dataset)과 시험 데이터셋(Test Dataset)으로 나누어(Split)줍니다.**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

6. **신경망 모델을 구축하고 컴파일(complie)한 후 학습합니다.**

이번 예제에서는 단층, 즉 **은닉층 없이 출력층으로만** 모델을 구성할 예정입니다.<br/>
그렇다면 입력층은 어디에 있을까요? 그리고 입력층의 노드 개수는 몇 개가 될까요?

In [ ]:
# Sequential: 레이어를 순차적으로 연결하여 신경망을 구성하는 방법

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

**`.compile`** 에서는 신경망에서 사용할 옵티마이저(**`optimizer`**), 손실 함수(**`loss`**), 지표(**`metrics`**)를 설정합니다.

In [ ]:
from tensorflow.keras.metrics import Precision

model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision()])

In [ ]:
# # .compile() : 신경망을 구성한 다음 신경망의 학습 방법을 결정해줍니다.

# model.compile(optimizer='sgd',
#               loss='binary_crossentropy',
#               metrics=['accuracy']) # 학습 지표를 정확도로 설정합니다.

**`.fit`** 은 실제로 신경망 학습이 진행되는 부분입니다.<br/>
에포크(`epochs`)를 조정하면 학습 횟수를 조정할 수 있습니다.

In [ ]:
model.fit(X_train, y_train, epochs=30)

7. **학습한 신경망 모델을 사용하여 평가합니다.**

In [ ]:
model.evaluate(X_test, y_test, verbose=2)

붓꽃 데이터 예제 코드에서

    > ❓ 1. 입력층의 노드 수는 몇 개일까요? (Hint : 입력 데이터의 특성 수)  
    > ❓ 2. **`Dense`** 층의 숫자는 왜 **1** 일까요?  
    > ❓ 3. **`Dense`** 층의 **`activation`** 은 왜 **`sigmoid`** 로 해주었을까요?  
    


### 손글씨 MNIST 예제



MNIST 손글씨 예제는 가장 기본적인 이미지 분류(Image Classification) 예제입니다.<br/>
사람이 **0-9까지 쓴 흑백 손글씨 숫자 이미지를 각 클래스로 분류**합니다.<br/>
데이터는 가로, 세로 28픽셀(pixels)로 구성된 정사각형 이미지입니다.

실제 데이터가 어떻게 생겼고 어떻게 분류되어야 할 지 이미지를 통해 알아보겠습니다.

<img src="https://abpaudel.com/assets/img/posts/mnist.png" width=500>

> ❓ *그렇다면 MNIST 예제는 이진 분류, 다중 분류, 회귀 중 어디에 속할까요?*  
> **<font color="ff6f61">항상 문제를 풀기 전에 자신이 풀고자 하는 문제가 어디에 속하는 지 생각</font>해보도록 합시다.**

1. **먼저 필요한 패키지와 라이브러리를 불러옵니다.**

In [ ]:
import pandas as pd
import tensorflow as tf

2. **데이터셋을 불러온 후 학습 데이터셋(Train Dataset)과 시험 데이터셋(Test Dataset)으로 나누어(Split)주고 픽셀값을 정규화 하여줍니다.**

In [ ]:
# keras에서 제공하는 예제 데이터셋을 불러옵니다.
mnist = tf.keras.datasets.mnist

# .load_data()를 통해 각 형태에 맞게 데이터를 불러올 수 있습니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

이미지 데이터에서는 정규화하는 과정이 중요합니다. 빼먹지 않도록 주의해주세요!

In [ ]:
# 이미지 데이터는 색의 명암을 0 ~ 255 사이의 숫자를 통해 표현합니다.
# 따라서 데이터가 0 ~ 255 사이의 숫자로 구성되어 있으니 이를 0 ~ 1 사이의 숫자로 normalization 해줍니다.

x_train, x_test = x_train / 255.0, x_test / 255.0

3. **레이블이 어떻게 구성되어 있는 지 확인해봅니다.**

    데이터의 레이블 구성 형태를 살펴봅니다.<br/>
    처음보는 데이터의 경우 데이터 자체를 디스플레이 하여 보면 도움이 됩니다.

In [ ]:
pd.unique(y_train)

4. **이제 본격적으로 신경망 모델을 구축해보겠습니다.**

이번 예제에서는 1개의 은닉층과 1개의 출력층으로만 모델을 구성할 예정입니다.<br/>
그렇다면 입력층의 노드 수는 몇 개가 될까요?

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# 위 코드와 동일한 역할을 수행하는 다른 코드입니다.
# 다른 사람의 코드를 보고 이해할 수 있도록 아래 방식에도 익숙해져 보도록 합시다.

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

이전 예제에서는 `Flatten` 함수가 없었는데요. 해당 함수는 왜 사용되는 것일까요?



In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

5. **학습한 신경망 모델을 사용하여 평가합니다.**

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

손글씨 MNIST 예제 코드에서

    > ❓ 1. 입력층의 노드 수는 몇 개일까요? (Hint : 입력 데이터의 특성 수)  
    > ❓ 2. `Flatten` 은 어떤 역할을 할까요?  
    > ❓ 3. 마지막 `Dense` 층의 숫자는 왜 10 일까요?  
    > ❓ 4. 마지막 `Dense` 층의 `activation` 은 왜 `softmax` 로 해주었을까요?  
   
